<a href="https://colab.research.google.com/github/bieri2/ATMS597_Project5_GroupB/blob/master/ATMS597_Project5_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import os
from datetime import datetime as dt

In [0]:
pip install metpy

In [0]:
from metpy.io import parse_metar_to_dataframe as parse_metar

In [0]:
# Define path to data
path  = '/content/drive/My Drive/Project5_Data/'
# List files in directory
files = os.listdir(path)

In [0]:
# Define function to read & parse a single file
def process_file(file):
    """
    Function to process an ASOS file downloaded from ftp://ftp.ncdc.noaa.gov/pub/data/asos-fivemin/.
    Reads a file and parses the METAR codes contained therein using MetPy. Data are parsed for 
    each time step contained in the file. Once this has been completed, the resulting DataFrames
    are concatenated into one DataFrame and returned.

    Parameters:
    -file (string): File name of the ASOS .dat file to be read and parsed

    Returns:
    -A pandas DataFrame containing the parsed data read from the ASOS file.
    """
    # Create empty list to hold data 
    all_data = []
    # Read data from file
    data = pd.read_csv(file)
    # Get month and year from file name
    time = dt.strptime(file[-10:-4], '%Y%m')
    print(time)
    # Parse METAR codes in each row (each row is a time step) using MetPy function
    for index, row in data.iterrows():
          try:
              all_data.append(parse_metar(row.values[0][52:], year = time.year, month =  time.month))
          # If error raised, just continue and exclude data for that time step
          except:
              print('Error at ' + str(index))
              print(row.values) 
              print('Excluding data for this timestep...')

    # Concatenate all DataFrames for each time step and return
    return pd.concat(all_data)

In [0]:
# Loop through all ASOS files 

# Define empty list to hold DataFrames
all_df = []
for file in files:
    print('Processing file ' + file)
    # Use function defined above to process data and append to list
    try:
        current = process_file(path + file)
        all_df.append(current)
    # If there's a problem with the file, exclude that file and keep going
    except:
        print('Problem with ' + file)
        print('Excluding data from this file...')

In [0]:
# Concatenate all DataFrames in list
all_data = pd.concat(all_data)

In [0]:
# Define the index
all_data.index = all_data.date_time

In [0]:
# Save to a .csv file
all_data.to_csv('/content/drive/My Drive/Project5_Data/all_data_KDSM_no_qc.csv')